# Workshop

In this workshop, we'll be building a podcast generator from a PDF file.

In [1]:
import os

os.environ["UNION_API_KEY"] = "..."  # enter the Union API key here

If you haven't already, register the `notebook_llama` tasks and workflows:

In [ ]:
# uncommend the line below to register the tasks and workflows if you haven't already
# !union register notebook_llama

In [2]:
import union
import rich

remote = union.UnionRemote(
    default_project="default",
    default_domain="development",
    interactive_mode_enabled=False,
)

def print_prompt(title: str, prompt: str):
    return rich.panel.Panel(prompt, title=title, width=120, border_style="red")

def print_text(title: str, text: str):
    return rich.panel.Panel(text, title=title, width=120, border_style="yellow")

## Extract Text from PDF

In [ ]:
from notebook_llama.preprocess_pdf import pdf_to_text, SYS_PROMPT as EXTRACT_PDF_SYS_PROMPT

rich.print(print_prompt("PDF Extraction System Prompt", EXTRACT_PDF_SYS_PROMPT))

pdf_to_text_execution = remote.execute(
    pdf_to_text,
    inputs={"pdf_path": "https://arxiv.org/pdf/2503.13571"},
)
pdf_to_text_execution

In [ ]:
pdf_to_text_execution = remote.wait(pdf_to_text_execution)
extracted_text = pdf_to_text_execution.outputs["o0"]

with open(extracted_text) as f:
    text = f.read()

print_text("Extracted Text", text)

## Write Transcript

In [ ]:
from notebook_llama.write_transcript import write_transcript, SYSTEM_PROMPT as WRITE_TRANSCRIPT_SYSTEM_PROMPT

rich.print(print_prompt("Write Transcript System Prompt", WRITE_TRANSCRIPT_SYSTEM_PROMPT))

write_transcript_execution = remote.execute(
    write_transcript,
    inputs={"pdf_text": extracted_text},
    version="latest",
)

write_transcript_execution

In [ ]:
write_transcript_execution = remote.wait(write_transcript_execution)
raw_transcript_file = write_transcript_execution.outputs["o0"]

with open(raw_transcript_file) as f:
    raw_transcript = f.read()

print_text("Raw Transcript", raw_transcript)

## Rewrite Transcript

In [ ]:
from notebook_llama.rewrite_transcript import rewrite_transcript, SYSTEM_PROMPT as REWRITE_TRANSCRIPT_SYSTEM_PROMPT

rich.print(print_prompt("Rewrite Transcript System Prompt", REWRITE_TRANSCRIPT_SYSTEM_PROMPT))

rewrite_transcript_execution = remote.execute(
    rewrite_transcript,
    inputs={"transcript": raw_transcript_file},
    version="latest",
)

rewrite_transcript_execution

In [ ]:
import json
import rich.markdown

rewrite_transcript_execution = remote.wait(rewrite_transcript_execution)
rewritten_transcript_file = rewrite_transcript_execution.outputs["o0"]

with open(rewritten_transcript_file) as f:
    rewritten_transcript = json.load(f)

text = ""
for speaker, line in rewritten_transcript:
    text += f"**{speaker}**: {line}\n\n"

print_text("Rewritten Transcript", rich.markdown.Markdown(text))

## Generate Podcast

In [ ]:
from notebook_llama.generate_podcast import generate_podcast, speaker1_description, speaker2_description

rich.print(print_prompt("Speaker 1 Description", speaker1_description))
rich.print(print_prompt("Speaker 2 Description", speaker2_description))

generate_podcast_execution = remote.execute(
    generate_podcast,
    inputs={"clean_transcript": rewritten_transcript_file},
    version="latest",
)

generate_podcast_execution

In [ ]:
from IPython.display import Audio

generate_podcast_execution = remote.wait(generate_podcast_execution)
podcast_file = generate_podcast_execution.outputs["o0"]

podcast_file.download()
Audio(podcast_file.path)
